# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame q q
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tralee,52.2704,-9.7026,10.14,93,20,3.09,IE,1735330328
1,1,kumasi,6.6885,-1.6244,27.09,22,24,1.46,GH,1735330329
2,2,nova sintra,14.8667,-24.7167,24.25,80,43,7.99,CV,1735330331
3,3,gisborne,-38.6533,178.0042,14.80,86,100,9.47,NZ,1735330332
4,4,petropavlovsk-kamchatsky,53.0452,158.6483,-3.67,54,75,7.00,RU,1735330333


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
print(city_data_df.columns)

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')


In [9]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='Humidity',
    colorbar=True,
    tiles='CartoLight',
    frame_width=1000,
    frame_height=750,
    title="City Humidity Map"
)
    

# Display the map
city_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,hawaiian paradise park,19.5933,-154.9731,25.73,74,0,2.06,US,1735330346
108,108,wailua homesteads,22.0669,-159.3780,25.44,85,0,1.54,US,1735330464
182,182,kailua-kona,19.6406,-155.9956,26.97,63,0,2.24,US,1735330553
207,207,lihue,21.9789,-159.3672,25.77,61,0,1.54,US,1735330584
250,250,holualoa,19.6228,-155.9522,24.90,63,0,1.64,US,1735330637


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = None

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
14,hawaiian paradise park,US,19.5933,-154.9731,74,None
108,wailua homesteads,US,22.0669,-159.3780,85,None
182,kailua-kona,US,19.6406,-155.9956,63,None
207,lihue,US,21.9789,-159.3672,61,None
250,holualoa,US,19.6228,-155.9522,63,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accomodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
kailua-kona - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
kapa'a - nearest hotel: No hotel found
sittwe - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
14,hawaiian paradise park,US,19.5933,-154.9731,74,No hotel found
108,wailua homesteads,US,22.0669,-159.3780,85,No hotel found
182,kailua-kona,US,19.6406,-155.9956,63,No hotel found
207,lihue,US,21.9789,-159.3672,61,No hotel found
250,holualoa,US,19.6228,-155.9522,63,No hotel found
277,saint-pierre,RE,-21.3393,55.4781,57,No hotel found
306,hilo,US,19.7297,-155.0900,70,No hotel found
371,kapa'a,US,22.0752,-159.3190,77,No hotel found
399,sittwe,MM,20.1500,92.9000,53,No hotel found
457,salalah,OM,17.0151,54.0924,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)